# Fine-tune Bert Model

In [ ]:
!pip3 install transformers

     |████████████████████████████████| 2.0MB 4.7MB/s 
     |████████████████████████████████| 3.2MB 21.2MB/s 
     |████████████████████████████████| 870kB 35.0MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.44-cp37-none-any.whl size=886084 sha256=f5d6d9219eede4e3d058e0dcd44b44ec76c3b3cc7fed765e29ef65d5767f6655
  Stored in directory: /root/.cache/pip/wheels/3e/fb/c0/13ab4d63d537658f448366744654323077c4d90069b6512f3c
Successfully built sacremoses


In [ ]:
import torch
import pandas as pd
import re


In [ ]:

# use GPU if available
if torch.cuda.is_available():    
    device = torch.device("cuda")
    print('Use GPU')
else:
    device = torch.device("cpu")

There are 1 GPU(s) available.
We will use the GPU: Tesla P100-PCIE-16GB


In [ ]:
def tokenize(text):

    # try:
    #     text = text.decode('utf-8').lower()
    # except:
    #     text = text.encode('utf-8').decode('utf-8').lower()
    text = text.lower()

    text = re.sub(u"\u2019|\u2018", "\'", text)
    text = re.sub(u"\u201c|\u201d", "\"", text)
    text = re.sub(r"http[s]?:[^\ ]+", " ", text)
    text = re.sub(r"&gt;", " ", text)
    text = re.sub(r"&lt;", " ", text)
    text = re.sub(r"&quot;", " ", text)
    text = re.sub(r"&nbsp;", " ", text)
    text = re.sub(r"via", " ", text)
    text = re.sub(r"#\ ", "#", text)
    text = re.sub(r"\\n", " ", text)
    text = re.sub(r"\\r", " ", text)
    text = re.sub(r"\\", " ", text)
    text = re.sub(r"[\(\)\[\]\{\}]", r" ", text)
    text = re.sub(r"#", " #", text)
    text = re.sub(r"\@", " ", text)
    text = re.sub(r"[\～\+\-\《\》\、\$\%\^\>\<\=\:\;\*\(\)\{\}\[\]\/\~\&\|\【\】]", " ", text)
    text = ' '.join(text.split())
    #words = text.split()
    return text


In [ ]:
trainset = pd.read_csv('train2.csv')
validset = pd.read_csv('valid2.csv')
testset = pd.read_csv('test.csv')

train_text = [tokenize(x) for x in trainset['text']]
valid_text = [tokenize(x) for x in validset['text']]
test_text = [tokenize(x) for x in testset['text']]

train_label = list(trainset.stars-1)
valid_label = list(validset.stars-1)


In [ ]:
from transformers import RobertaTokenizer
# Load the BERT tokenizer.
print('Loading Roberta tokenizer...')
tokenizer = RobertaTokenizer.from_pretrained('roberta-base', do_lower_case=True)

Loading Roberta tokenizer...


In [ ]:
# Tokenize all of the sentences and map the tokens to thier word IDs.
train_input_ids = []
# For every sentence...
for sent in train_text:

    encoded_sent = tokenizer.encode(
                        sent,                      # Sentence to encode.
                        add_special_tokens = True, # Add '[CLS]' and '[SEP]'
                        
                   )
    
    # Add the encoded sentence to the list.
    train_input_ids.append(encoded_sent)


Token indices sequence length is longer than the specified maximum sequence length for this model (577 > 512). Running this sequence through the model will result in indexing errors


Original:  "what do you do for recreation?" "oh, the usual. i bowl. drive around. the occasional acid flashback." i do not actually like bowling. it's my nature. i only like things that i am good at and i am not even certain i would like to be good at bowling so i had my reservations when being invited to hang with the optimism club kids but i love these kids so... "fuck it, dude, let's go bowling." i do have to admit i really had a blast here. it is vintage and scummy on the outside which to me is just a bowling alley being honest with itself. it looks so old and dingy from the outside that you are slightly taken aback by the automatic doors. it's a bit like you have never seen one before and wow isn't technology amazing? i have to say the bar service was a little slow. they rotated between three bartenders and 2 3 of them were ok so i will let it slide. the third on the other hand was probably better suited for my cube monkey, paper pushing job. the beers were however 2 for a domesti

In [ ]:
print('Max sentence length: ', max([len(sen) for sen in train_input_ids]))

Max sentence length:  1223


In [ ]:
from keras.preprocessing.sequence import pad_sequences
# Set the maximum sequence length.

MAX_LEN = 256
print('\nPadding/truncating all sentences to %d values...' % MAX_LEN)
print('\nPadding token: "{:}", ID: {:}'.format(tokenizer.pad_token, tokenizer.pad_token_id))
# Pad our input tokens with value 0.

train_input_ids = pad_sequences(train_input_ids, maxlen=MAX_LEN, dtype="long", 
                          value=0, truncating="post", padding="post")
print('\Done.')


Padding/truncating all sentences to 256 values...

Padding token: "<pad>", ID: 1
\Done.


In [ ]:
# Create attention masks
train_attention_masks = []
# For each sentence...
for sent in train_input_ids:
    
    # Create the attention mask.
    #   - If a token ID is 0, then it's padding, set the mask to 0.
    #   - If a token ID is > 0, then it's a real token, set the mask to 1.
    att_mask = [int(token_id > 0) for token_id in sent]
    
    # Store the attention mask for this sentence.
    train_attention_masks.append(att_mask)

In [ ]:
# Tokenize all of the sentences and map the tokens to thier word IDs.
valid_input_ids = []
for sent in valid_text:

    encoded_sent = tokenizer.encode(
                        sent,                      # Sentence to encode.
                        add_special_tokens = True, # Add '[CLS]' and '[SEP]'
                        
                   )
    
    # Add the encoded sentence to the list.
    valid_input_ids.append(encoded_sent)

print('Max sentence length: ', max([len(sen) for sen in valid_input_ids]))

from keras.preprocessing.sequence import pad_sequences

MAX_LEN = 256
print('\nPadding/truncating all sentences to %d values...' % MAX_LEN)
print('\nPadding token: "{:}", ID: {:}'.format(tokenizer.pad_token, tokenizer.pad_token_id))
# Pad our input tokens with value 0.

valid_input_ids = pad_sequences(valid_input_ids, maxlen=MAX_LEN, dtype="long", 
                          value=0, truncating="post", padding="post")
print('\Done.')

# Create attention masks
valid_attention_masks = []
# For each sentence...
for sent in valid_input_ids:
    
    # Create the attention mask.
    #   - If a token ID is 0, then it's padding, set the mask to 0.
    #   - If a token ID is > 0, then it's a real token, set the mask to 1.
    att_mask = [int(token_id > 0) for token_id in sent]
    
    # Store the attention mask for this sentence.
    valid_attention_masks.append(att_mask)


Original:  food is sometimes sometimes great and somethings just good. wasn't a fan of their brunch but lunch and dinner is good. service is hit and miss.
Token IDs: [0, 13193, 16, 2128, 2128, 372, 8, 45420, 298, 1033, 95, 205, 4, 938, 75, 10, 2378, 9, 49, 25003, 53, 4592, 8, 3630, 16, 205, 4, 544, 16, 478, 8, 2649, 4, 2]
Max sentence length:  1108

Padding/truncating all sentences to 256 values...

Padding token: "<pad>", ID: 1
\Done.


In [ ]:
train_inputs, valid_inputs = train_input_ids, valid_input_ids
train_masks, valid_masks = train_attention_masks, valid_attention_masks
train_labels, valid_labels = train_label, valid_label

In [ ]:
# Convert all inputs and labels into torch tensors, the required datatype 
# for our model.
train_inputs = torch.tensor(train_inputs)
validation_inputs = torch.tensor(valid_inputs)
train_labels = torch.tensor(train_labels)
validation_labels = torch.tensor(valid_labels)
train_masks = torch.tensor(train_masks)
validation_masks = torch.tensor(valid_masks)

In [ ]:
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler

batch_size = 32
# Create the DataLoader for our training set.
train_data = TensorDataset(train_inputs, train_masks, train_labels)
train_sampler = RandomSampler(train_data)
train_dataloader = DataLoader(train_data, sampler=train_sampler, batch_size=batch_size)
# Create the DataLoader for our validation set.
validation_data = TensorDataset(validation_inputs, validation_masks, validation_labels)
validation_sampler = SequentialSampler(validation_data)
validation_dataloader = DataLoader(validation_data, sampler=validation_sampler, batch_size=batch_size)

In [ ]:
from transformers import RobertaForSequenceClassification, AdamW, RobertaConfig
# Load BertForSequenceClassification, the pretrained BERT model with a single 
# linear classification layer on top. 
model = RobertaForSequenceClassification.from_pretrained(
    "roberta-base", # Use the 12-layer BERT model, with an uncased vocab.
    num_labels = 5, # The number of output labels--2 for binary classification.
                    # You can increase this for multi-class tasks.   
    output_attentions = False, # Whether the model returns attentions weights.
    output_hidden_states = False, # Whether the model returns all hidden-states.
)
# Tell pytorch to run this model on the GPU.
model.cuda()

Some weights of the model checkpoint at roberta-base were not used when initializing RobertaForSequenceClassification: ['lm_head.bias', 'lm_head.dense.weight', 'lm_head.dense.bias', 'lm_head.layer_norm.weight', 'lm_head.layer_norm.bias', 'lm_head.decoder.weight', 'roberta.pooler.dense.weight', 'roberta.pooler.dense.bias']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at roberta-base and are newly initialized: ['classifier.dense.weight', 'classifie

RobertaForSequenceClassification(
  (roberta): RobertaModel(
    (embeddings): RobertaEmbeddings(
      (word_embeddings): Embedding(50265, 768, padding_idx=1)
      (position_embeddings): Embedding(514, 768, padding_idx=1)
      (token_type_embeddings): Embedding(1, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): RobertaEncoder(
      (layer): ModuleList(
        (0): RobertaLayer(
          (attention): RobertaAttention(
            (self): RobertaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): RobertaSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerN

In [ ]:
# Get all of the model's parameters as a list of tuples.
params = list(model.named_parameters())
print('The BERT model has {:} different named parameters.\n'.format(len(params)))
print('==== Embedding Layer ====\n')
for p in params[0:5]:
    print("{:<55} {:>12}".format(p[0], str(tuple(p[1].size()))))
print('\n==== First Transformer ====\n')
for p in params[5:21]:
    print("{:<55} {:>12}".format(p[0], str(tuple(p[1].size()))))
print('\n==== Output Layer ====\n')
for p in params[-4:]:
    print("{:<55} {:>12}".format(p[0], str(tuple(p[1].size()))))

The BERT model has 201 different named parameters.

==== Embedding Layer ====

roberta.embeddings.word_embeddings.weight               (50265, 768)
roberta.embeddings.position_embeddings.weight             (514, 768)
roberta.embeddings.token_type_embeddings.weight             (1, 768)
roberta.embeddings.LayerNorm.weight                           (768,)
roberta.embeddings.LayerNorm.bias                             (768,)

==== First Transformer ====

roberta.encoder.layer.0.attention.self.query.weight       (768, 768)
roberta.encoder.layer.0.attention.self.query.bias             (768,)
roberta.encoder.layer.0.attention.self.key.weight         (768, 768)
roberta.encoder.layer.0.attention.self.key.bias               (768,)
roberta.encoder.layer.0.attention.self.value.weight       (768, 768)
roberta.encoder.layer.0.attention.self.value.bias             (768,)
roberta.encoder.layer.0.attention.output.dense.weight     (768, 768)
roberta.encoder.layer.0.attention.output.dense.bias           (

In [ ]:
# Note: AdamW is a class from the huggingface library (as opposed to pytorch) 
# I believe the 'W' stands for 'Weight Decay fix"
optimizer = AdamW(model.parameters(),
                  lr = 0.000015, # args.learning_rate - default is 5e-5, our notebook had 2e-5
                  eps = 1e-8 # args.adam_epsilon  - default is 1e-8.
                )
from transformers import get_linear_schedule_with_warmup
# Number of training epochs (authors recommend between 2 and 4)
epochs = 3
# Total number of training steps is number of batches * number of epochs.
total_steps = len(train_dataloader) * epochs
# Create the learning rate scheduler.
scheduler = get_linear_schedule_with_warmup(optimizer, 
                                            num_warmup_steps = 0, # Default value in run_glue.py
                                            num_training_steps = total_steps)

In [ ]:
import numpy as np
# Function to calculate the accuracy of our predictions vs labels
def flat_accuracy(preds, labels):
    pred_flat = np.argmax(preds, axis=1).flatten()
    labels_flat = labels.flatten()
    return np.sum(pred_flat == labels_flat) / len(labels_flat)

In [ ]:
import time
import datetime
def format_time(elapsed):
    '''
    Takes a time in seconds and returns a string hh:mm:ss
    '''
    # Round to the nearest second.
    elapsed_rounded = int(round((elapsed)))
    
    # Format as hh:mm:ss
    return str(datetime.timedelta(seconds=elapsed_rounded))

In [ ]:

loss_values = []
for epoch_i in range(0, epochs):
    
    # ========================================
    #               Training
    # ========================================
    
    # Perform one full pass over the training set.
    print("")
    print('======== Epoch {:} / {:} ========'.format(epoch_i + 1, epochs))
    print('Training...')
    # Measure how long the training epoch takes.
    t0 = time.time()
    # Reset the total loss for this epoch.
    total_loss = 0
    
    model.train()
    # For each batch of training data...
    for step, batch in enumerate(train_dataloader):
        # Progress update every 40 batches.
        if step % 40 == 0 and not step == 0:
            # Calculate elapsed time in minutes.
            elapsed = format_time(time.time() - t0)
            
            # Report progress.
            print('  Batch {:>5,}  of  {:>5,}.    Elapsed: {:}.'.format(step, len(train_dataloader), elapsed))
        
        b_input_ids = batch[0].to(device)
        b_input_mask = batch[1].to(device)
        b_labels = batch[2].to(device)
        
        model.zero_grad()        
        
        outputs = model(b_input_ids, 
                    token_type_ids=None, 
                    attention_mask=b_input_mask, 
                    labels=b_labels)
        
        loss = outputs[0]
        if step % 20 == 19:
          print(loss)
        
        total_loss += loss.item()
        # Perform a backward pass to calculate the gradients.
        loss.backward()
        # Clip the norm of the gradients to 1.0.
        torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)
        
        optimizer.step()
        # Update the learning rate.
        scheduler.step()
    # Calculate the average loss over the training data.
    avg_train_loss = total_loss / len(train_dataloader)            
    
    # Store the loss value for plotting the learning curve.
    loss_values.append(avg_train_loss)
    print("")
    print("  Average training loss: {0:.2f}".format(avg_train_loss))
    print("  Training epcoh took: {:}".format(format_time(time.time() - t0)))
        
    # ========================================
    #               Validation
    # ========================================
    # After the completion of each training epoch, measure our performance on
    # our validation set.
    print("")
    print("Running Validation...")
    t0 = time.time()
    # Put the model in evaluation mode--the dropout layers behave differently
    # during evaluation.
    model.eval()
    # Tracking variables 
    eval_loss, eval_accuracy = 0, 0
    nb_eval_steps, nb_eval_examples = 0, 0
    # Evaluate data for one epoch
    for batch in validation_dataloader:
        
        # Add batch to GPU
        batch = tuple(t.to(device) for t in batch)
        
        # Unpack the inputs from our dataloader
        b_input_ids, b_input_mask, b_labels = batch
        
        
        with torch.no_grad():        
            
            outputs = model(b_input_ids, 
                            token_type_ids=None, 
                            attention_mask=b_input_mask)
        
        
        logits = outputs[0]

        logits = logits.detach().cpu().numpy()
        label_ids = b_labels.to('cpu').numpy()
        
        # Calculate the accuracy for this batch of test sentences.
        tmp_eval_accuracy = flat_accuracy(logits, label_ids)
        
        # Accumulate the total accuracy.
        eval_accuracy += tmp_eval_accuracy
        # Track the number of batches
        nb_eval_steps += 1
        
    # Report the final accuracy for this validation run.
    print("  Accuracy: {0:.2f}".format(eval_accuracy/nb_eval_steps))
    print("  Validation took: {:}".format(format_time(time.time() - t0)))
print("")
print("Training complete!")


======== Epoch 1 / 3 ========
Training...
tensor(1.4989, device='cuda:0', grad_fn=<NllLossBackward>)
tensor(1.5424, device='cuda:0', grad_fn=<NllLossBackward>)
  Batch    40  of    625.    Elapsed: 0:00:31.
tensor(1.1875, device='cuda:0', grad_fn=<NllLossBackward>)
tensor(1.1823, device='cuda:0', grad_fn=<NllLossBackward>)
  Batch    80  of    625.    Elapsed: 0:01:02.
tensor(0.8620, device='cuda:0', grad_fn=<NllLossBackward>)
tensor(0.9292, device='cuda:0', grad_fn=<NllLossBackward>)
  Batch   120  of    625.    Elapsed: 0:01:33.
tensor(0.8782, device='cuda:0', grad_fn=<NllLossBackward>)
tensor(0.6182, device='cuda:0', grad_fn=<NllLossBackward>)
  Batch   160  of    625.    Elapsed: 0:02:03.
tensor(0.6357, device='cuda:0', grad_fn=<NllLossBackward>)
tensor(0.5418, device='cuda:0', grad_fn=<NllLossBackward>)
  Batch   200  of    625.    Elapsed: 0:02:34.
tensor(0.6656, device='cuda:0', grad_fn=<NllLossBackward>)
tensor(0.9106, device='cuda:0', grad_fn=<NllLossBackward>)
  Batch   240 

In [ ]:
# make predictions on test set
test_input_ids = []
for sent in test_text:
    
    encoded_sent = tokenizer.encode(
                        sent,                      # Sentence to encode.
                        add_special_tokens = True, # Add '[CLS]' and '[SEP]'
                   )
    
    test_input_ids.append(encoded_sent)
# Pad our input tokens
test_input_ids = pad_sequences(test_input_ids, maxlen=MAX_LEN, 
                          dtype="long", truncating="post", padding="post")
# Create attention masks
test_attention_masks = []
# Create a mask of 1s for each token followed by 0s for padding
for seq in test_input_ids:
  seq_mask = [float(i>0) for i in seq]
  test_attention_masks.append(seq_mask) 
# Convert to tensors.
prediction_inputs = torch.tensor(test_input_ids)
prediction_masks = torch.tensor(test_attention_masks)
prediction_labels = torch.tensor([0]*len(test_input_ids))
# Set the batch size.  
batch_size = 32  
# Create the DataLoader.
prediction_data = TensorDataset(prediction_inputs, prediction_masks, prediction_labels)
prediction_sampler = SequentialSampler(prediction_data)
prediction_dataloader = DataLoader(prediction_data, sampler=prediction_sampler, batch_size=batch_size)

In [ ]:
# Prediction on test set
print('Predicting labels for {:,} test sentences...'.format(len(prediction_inputs)))
# Put model in evaluation mode
model.eval()
# Tracking variables 
predictions , true_labels = [], []
# Predict 
for batch in prediction_dataloader:
  batch = tuple(t.to(device) for t in batch)
  
  b_input_ids, b_input_mask, b_labels = batch
  

  with torch.no_grad():
      # Forward pass, calculate logit predictions
      outputs = model(b_input_ids, token_type_ids=None, 
                      attention_mask=b_input_mask)
  logits = outputs[0]
  # Move logits and labels to CPU
  logits = logits.detach().cpu().numpy()
  # label_ids = b_labels.to('cpu').numpy()
  logits = np.argmax(logits, axis=1).flatten()
  # Store predictions and true labels
  predictions.append(logits+1)
  # true_labels.append(label_ids)
print('DONE.')

Predicting labels for 2,000 test sentences...
DONE.


In [ ]:
import plotly.express as px
f = pd.DataFrame(loss_values)
f.columns=['Loss']
fig = px.line(f, x=f.index, y=f.Loss)
fig.update_layout(title='Training loss of the Model',
                   xaxis_title='Epoch',
                   yaxis_title='Loss')
fig.show()

In [ ]:
ans = list(predictions[0])
for i in predictions[1:]:
  ans.extend(list(i))


In [1]:
(pd.DataFrame({'review_id':testset.review_id, 'stars':ans})).to_csv('test_sample.csv')

NameError: ignored